# Lesson 0 - Getting Started

## Overview

In this lesson, we will run a hands-on calibration experiment with the Poconos watershed. Specifically, we will be calibrating the basin using observed and modeled streamflow at gage: 

<<01447720. This calibration exercise will focus on a short duration in September, 2011 coinciding with Hurricane Irene. We will be running a 25 iteration calibration exercise. >> * to be edited *

The calibration procedure relies on a python library named PyWrfHydroCalib, which explicitly handles the setup, model tuning, and provides diagnostics supporting the calibration process. This lesson is intended to provide a 10,000ft overview of the calibration process, system requirements, and workflow.



# What Is It? Defining Key Terms:
#### WRF-Hydro: 
An open-source community model, is used for a range of projects, including flash flood prediction, regional hydroclimate impacts assessment, seasonal forecasting of water resources, and land-atmosphere coupling studies. 

#### The National Water Model (NWM):
A joint development effort between the National Center for Atmospheric Research (NCAR) and the National Weather Service (NWS) Office of Water Prediction (OWP). The NWM is a set of operational analysis and forecast configurations of the WRF-Hydro modeling system (Gochis et al., 2013) which runs over the entire continental United States (https://water.noaa.gov/about/nwm). 

#### PyWrfHydroCalib:
A robust workflow for calibrating parameters over individual basins occurred with a focus on automating the process for hundreds of forecast points in the NWM domain. This document set of exercises describe the various steps involved in setting up a WRF-Hydro calibration experiment, along with how to execute the calibration workflow and access the various results from it. This version of the calibration workflow has been designed to calibrate against observed streamflow. However, future iterations will expand the focus to calibrate against other observed hydrologic states, such as snow, inundation, and soil moisture.


# System Requirements

### Background Knowledge:
Calibration is an advanced topic that goes beyond setting up the NWM or WRF-Hydro for a model simulation. Some understanding of the preparation of datasets, hydrologic parameters, and their associated impact on model states is needed. You will decide which parameters to calibrate, along with the proper ranges, domain size, and the length of the calibration simulation/evaluation period. It is highly encouraged that you run sensitivity analysis for the modeling domain to explore which parameters have significant impact on the modeled states being calibrated.


### Software and conventions
#### Python Packages:
To run the calibration workflow, there are multiple software requirements and pre-processing steps that need to take place prior to actual calibration. Currently, the calibration workflow relies on a set of Python and R libraries to execute various functions and plotting capabilities. This is in addition to existing requirements to compile and run WRF-Hydro/NWM. As of early 2019, the calibration workflow code has been migrated from Python 2 to Python 3. The following Python libraries are used by the workflow:

1.) numpy
2.) NetCDF4 (for Python)
3.) pandas
4.) psutil
5.) psycopg2


In [ ]:
# Optional: Verify Python Libraries
import numpy
import netCDF4
import pandas
import psutil
import psycopg2
print('all python packages accounted for')

#### R Libraries
For R, there are several additional libraries required for proper analysis and plotting to take place. In the future, functions in R will be migrated over to Python to reduce dependencies. The following R libraries are needed for proper execution:

1.) data.table
2.) ggplot2
3.) ncdf4
4.) plyr
5.) boot
6.) sensitivity
7.) randtoolbox
8.) gridExtra
9.) hydroGOF

** Further information on code requirements, and links to required packages can be found within the User Manual.

In [5]:
# Verify R libraries (for Jupyter notebook on Cheyenne, switch to R kernel for now, will be updated for docker notebook)
library(data.table)
library(ggplot2)
library(ncdf4)
library(plyr)
library(boot)
library(sensitivity)
library(randtoolbox)
library(gridExtra)
library(hydroGOF)

ERROR: Error in library(data.table): there is no package called ‘data.table’


### Prerequisites:
1] Installation of the WRF-Hydro/NWM modeling system (use a version that is pointed out in the release notes of the package) See the 'WRF-Hydro Technical Description and User Guide' available from the WRF-Hydro Modeling System website for details: https://ral.ucar.edu/projects/wrf_hydro.

2**] Preparation of a modeling domain with a forecast streamflow point

3**] Preparation of associated forcing files necessary to run the model simulations

4**] Preparation of input observation files that will be used in the calibration workflow. Further detail regarding required input files can be found in the PyWrfHydroCalib can be found in the User Manual. 

** --> All of these files are provided for you in this training. Preparation of these files are detailed in supplemental lessons S1,S2,S3

### Additional Files (Mandatory):

In addition to compiling the WRF-Hydro code, there are a set of parameter NetCDF files that are generated from the GIS pre-processor (see WRF-Hydro user guides and documentation) that are modified during the calibration workflow. The following parameter files from the output of the WRF-Hydro GIS Preprocessing tool, which are unique to each modeling domain, are needed in addition to the regular input files for the calibration:

*soil_properties.nc* :
A 2D soil property file that contains values impacting sub-surface hydrologic response.

*Fulldom.nc* :
The 2D geospatial fabric utilized for the high-resolution routing.

*GWBUCKPARM.nc* : 
The 2D NetCDF groundwater bucket parameter file.

*HYDRO_TBL_2D.nc* :
The 2D NetCDF of surface hydrologic parameters impacting hydrologic response for overland flow routing.

*CHANPARM.TBL* :
The table file containing the channel properties in case the user is calibrating the channel properties using the CHANPARM.TBL


In [ ]:
# LINUX SHELL:
#<Display path to Domain & input dirs, showing input paths -- update once container established>

In [6]:
%%bash
ls /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/example_case/Calibration/Input_Files/01447720/

FORCING
Fulldom.nc
geo_em.nc
GEOGRID_LDASOUT_Spatial_Metadata.nc
GWBUCKPARM.nc
HYDRO_TBL_2D.nc
LAKEPARM.nc
OBS
params.txt
RouteLink.nc
script_forcing_subset.txt
script_forcing_subset.txt_NWM_REALTIME
soil_properties.nc
spatialweights.nc
wrfinput.nc


### Navigating the Example File Structure:

Within the main Directory, there are 3 subdirectories:

In [10]:
%%bash
cd /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/
ls

example_case
PyWrfHydroCalib
wrf_hydro_nwm_public


example_case -- contains all domain and input files:


In [17]:
%%bash
cd /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/example_case/Calibration/Input_Files/01447720/
ls

FORCING
Fulldom.nc
geo_em.nc
GEOGRID_LDASOUT_Spatial_Metadata.nc
GWBUCKPARM.nc
HYDRO_TBL_2D.nc
LAKEPARM.nc
OBS
params.txt
RouteLink.nc
script_forcing_subset.txt
script_forcing_subset.txt_NWM_REALTIME
soil_properties.nc
spatialweights.nc
wrfinput.nc


PyWrfHydroCalib  -- contains calibration scripts, and user setup files

In [18]:
%%bash
cd /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/PyWrfHydroCalib/
ls

calibOrchestrator.py
calib.py
core
initDB.py
inputDomainMeta.py
jobInit.py
LICENSE.md
PyWrfHydroCalib_User_Guide.pdf
README.md
runValidOrchestrator.py
sensitivity.py
setup_files
spinOrchestrator.py
spinup.py
util
validation.py
validOrchestrator.py


In [19]:
%%bash
cd /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/PyWrfHydroCalib/setup_files/
ls

calib_params.tbl
domainMetaTemplate.csv
gage_list_template.csv
sens_params.tbl
setup.parm


wrf_hydro_nwm_public -- contains wrf-hydro code repository

In [22]:
%%bash
cd /glade/work/arezoo/RFC_Training/RFC_CalibTrain2021/wrf_hydro_nwm_public/
ls

CODE_OF_CONDUCT.md
LICENSE.txt
NEWS.md
README.md
tests
trunk


## Conclusion:

Once all Input and Setup files are prepared, and verified, we can begin the process of setting up a calibration. Proceed to *Lesson 1 - Calibration Overview*

In [ ]:
### DEV_ END